# **DIC driven FE interpretation of a shear zone**

This notebook shows how to derive a deeper interpretation of the DIC displacements measured on a grid.
Based on an FE-mesh, it delivers the strain, stress and damage tensors. Further processing of the 
anisotropic damage tensor is used to detect the localized cracks.

In [1]:
%matplotlib widget

In [2]:
import ibvpy.api as ib
import matplotlib.pylab as plt
from mayavi import mlab
from bmcs_shear.api import CrackBridgeAdv
from bmcs_shear.dic_crack import \
    DICGrid, DICAlignedGrid, DICCOR, DICStateFields
from bmcs_shear.dic_crack.dic_state_fields import DICStateFields
from matplotlib import cm
import numpy as np
np.seterr(divide='ignore', invalid='ignore');

# DIC grid

## Input displacements on a grid

In [3]:
dic_grid = DICGrid(start_t=0, end_t=1, U_factor=100, dir_name='B1_TV1')
dic_grid.sz_bd.Rectangle = True
dic_grid.sz_bd.matrix_.trait_set(f_t=0.3 * 38**(2/3), f_c=38, d_a=16, E_c=28000)
dic_grid.sz_bd.trait_set(H=350, B=320, L=1500)
dic_grid.sz_bd.csl.add_layer(CrackBridgeAdv(z=50, n=5, d_s=16, E=210000, f_c=38))

In [4]:
dic_grid = DICGrid(start_t = 0, end_t =26, L = 1500, U_factor = 1, dir_name='B1_TV2')
dic_grid.sz_bd.Rectangle = True
dic_grid.sz_bd.matrix_.trait_set(f_t = 0.3 * 38 **(2/3), f_c = 38, d_a = 16, E_c = 28000) 
dic_grid.sz_bd.trait_set(H=350, B=320, L=1500)
dic_grid.sz_bd.csl.add_layer(CrackBridgeAdv(z=50, n=5, d_s=16, E = 210000, f_c = 38))

In [9]:
dic_grid = DICGrid(start_t=0, end_t=1, U_factor=100, dir_name='B10_TV2')

## Finite element processing of the displacement field

In [10]:
import traits.api as tr
from scipy.signal import argrelextrema
class DICSFDebug(DICStateFields):
    pass


In [11]:
#dsf = DICStateFields(dic_grid=dic_grid, t=0.5)
dsf = DICSFDebug(dic_grid=dic_grid, t=0.5)
dsf.tmodel_.trait_set(E=5000, c_T=0, nu=0.18, epsilon_0=0.0005, epsilon_f=0.01);
dsf.eval()

# State fields on a regular grid

In [12]:
dsf.interact()

(28, 7) (28, 7) [ 5 18 16  4 15 14  1] (28, 7)


# TODO - in the `DICStateFields` class

 - decompose the fields into properties

# Next steps in crack detection

 - identify the crack tip 
 - identify the index regions between cracks in the final stage (reference frame -- and rotating frame)
 - improve the detection of the cracks at the bottom - avoid multiple cracks next to each other
 - incremental analysis - backward and forward crack processing
 - exploit the principle damage direction to limit the range of the crack increment
 - how to detect the start of the failure crack - this would only work for incremental evaluation?

# Detection provided 

Along the detected crack, process the crack opening and sliding starting from the tip.

With the fixed and rotating frames detected - process the COR histories 

Derive the moment - rotation relation for each crack

Use the rigid rotation kinematics to evaluate the opening and sliding - compare with the experimental values

Apply the values needed for comparison 